In [1]:
import tensorflow_decision_forests as tfdf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

2023-07-26 18:48:45.235267: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
normal = pd.read_csv("final-normal-data-set.csv", nrows=1000)
anormal = pd.read_csv("final-anormal-data-set.csv", nrows=1000)

#inserting column for result 
normal.insert(82, 'result', 0)
anormal.insert(82, 'result', 1)

In [4]:
#combining datasets
frames = [normal, anormal]
df = pd.concat(frames)

In [5]:
def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

1367 examples in training, 633 examples for testing.


In [6]:
label = "result"
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label)

In [7]:
# random forest
model_1 = tfdf.keras.RandomForestModel(verbose=2)

# Train the model.
model_1.fit(train_ds)

Use 8 thread(s) for training
Use /var/folders/wx/r87f739d16n6_b4_57_dnxdw0000gn/T/tmpckarieso as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'cpu_guest': <tf.Tensor 'data:0' shape=(None,) dtype=float64>, 'cpu_guest_nice': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'cpu_idle': <tf.Tensor 'data_2:0' shape=(None,) dtype=float64>, 'cpu_iowait': <tf.Tensor 'data_3:0' shape=(None,) dtype=float64>, 'cpu_irq': <tf.Tensor 'data_4:0' shape=(None,) dtype=float64>, 'cpu_nice': <tf.Tensor 'data_5:0' shape=(None,) dtype=float64>, 'cpu_softirq': <tf.Tensor 'data_6:0' shape=(None,) dtype=float64>, 'cpu_steal': <tf.Tensor 'data_7:0' shape=(None,) dtype=float64>, 'cpu_system': <tf.Tensor 'data_8:0' shape=(None,) dtype=float64>, 'cpu_total': <tf.Tensor 'data_9:0' shape=(None,) dtype=float64>, 'cpu_user': <tf.Tensor 'data_10:0' shape=(None,) dtype=float64>, 'diskio_sda1_disk_name': <tf.Tensor 'data_11:0' shape=(None,) dtype=string>, 'diskio_sda1_k

[INFO 23-07-25 20:59:45.6393 PDT kernel.cc:773] Start Yggdrasil model training
[INFO 23-07-25 20:59:45.6404 PDT kernel.cc:774] Collect training examples
[INFO 23-07-25 20:59:45.6405 PDT kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 23-07-25 20:59:45.6413 PDT kernel.cc:393] Number of batches: 2
[INFO 23-07-25 20:59:45.6414 PDT kernel.cc:394] Number of examples: 1367
[INFO 23-07-25 20:59:45.6450 PDT data_spec_inference.cc:305] 1359 item(s) have been pruned (i.e. they are considered out of dictionary) for the column timestamp (0 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
[INFO 23-07-25 20:59:45.6462 PDT kernel.cc:7

Model trained in 0:00:00.165483
Compiling model...
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


In [8]:
model_1.compile(metrics=["accuracy"])
evaluation = model_1.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

1/1 [==============================] - 1s 707ms/step - loss: 0.0000e+00 - accuracy: 1.0000

loss: 0.0000
accuracy: 1.0000


In [100]:
# nvidia dataset

df_train = pd.read_json("nvidia_train.json")
df_test = pd.read_csv("nvidia_test.csv")
df_train.head()

,nvidia_smi_log.timestamp,nvidia_smi_log.gpu.pci.tx_util,nvidia_smi_log.gpu.pci.rx_util,nvidia_smi_log.gpu.fb_memory_usage.used,nvidia_smi_log.gpu.fb_memory_usage.free,nvidia_smi_log.gpu.bar1_memory_usage.total,nvidia_smi_log.gpu.bar1_memory_usage.used,nvidia_smi_log.gpu.bar1_memory_usage.free,nvidia_smi_log.gpu.utilization.gpu_util,nvidia_smi_log.gpu.utilization.memory_util,...,nvidia_smi_log.gpu.applications_clocks.graphics_clock,nvidia_smi_log.gpu.applications_clocks.mem_clock,nvidia_smi_log.gpu.default_applications_clocks.graphics_clock,nvidia_smi_log.gpu.default_applications_clocks.mem_clock,nvidia_smi_log.gpu.max_clocks.graphics_clock,nvidia_smi_log.gpu.max_clocks.sm_clock,nvidia_smi_log.gpu.max_clocks.mem_clock,nvidia_smi_log.gpu.max_clocks.video_clock,nvidia_smi_log.gpu.max_customer_boost_clocks.graphics_clock,label
0,Wed Mar 3 15:03:01 2021,0,0,3909,12251,16384,7,16377,100,2,...,1530,877,1312,877,1530,1530,877,1372,1530,0
1,Wed Mar 3 15:04:01 2021,0,2000,3909,12251,16384,7,16377,100,5,...,1530,877,1312,877,1530,1530,877,1372,1530,0
2,Wed Mar 3 15:05:01 2021,1000,2000,3909,12251,16384,7,16377,100,5,...,1530,877,1312,877,1530,1530,877,1372,1530,0
3,Wed Mar 3 15:06:01 2021,0,2000,3909,12251,16384,7,16377,100,6,...,1530,877,1312,877,1530,1530,877,1372,1530,0
4,Wed Mar 3 15:07:01 2021,1000,2000,3909,12251,16384,7,16377,100,6,...,1530,877,1312,877,1530,1530,877,1372,1530,0


In [96]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [84]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(df_train, label="label")

label = "mining"

classes = df_test[label].unique().tolist()
print(f"Label classes: {classes}")

df_test[label] = df_test[label].map(classes.index)

test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(df_test, label="mining")

Label classes: [False, True]


In [97]:
# Specify the model.
model_1 = tfdf.keras.RandomForestModel(verbose=2)

# Train the model.
model_1.fit(train_ds)

Use 8 thread(s) for training
Use /var/folders/wx/r87f739d16n6_b4_57_dnxdw0000gn/T/tmp84xv4qki as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'nvidia_smi_log.timestamp': <tf.Tensor 'data:0' shape=(None,) dtype=string>, 'nvidia_smi_log.gpu.pci.tx_util': <tf.Tensor 'data_1:0' shape=(None,) dtype=int64>, 'nvidia_smi_log.gpu.pci.rx_util': <tf.Tensor 'data_2:0' shape=(None,) dtype=int64>, 'nvidia_smi_log.gpu.fb_memory_usage.used': <tf.Tensor 'data_3:0' shape=(None,) dtype=int64>, 'nvidia_smi_log.gpu.fb_memory_usage.free': <tf.Tensor 'data_4:0' shape=(None,) dtype=int64>, 'nvidia_smi_log.gpu.bar1_memory_usage.total': <tf.Tensor 'data_5:0' shape=(None,) dtype=int64>, 'nvidia_smi_log.gpu.bar1_memory_usage.used': <tf.Tensor 'data_6:0' shape=(None,) dtype=int64>, 'nvidia_smi_log.gpu.bar1_memory_usage.free': <tf.Tensor 'data_7:0' shape=(None,) dtype=int64>, 'nvidia_smi_log.gpu.utilization.gpu_util': <tf.Tensor 'data_8:0' shape=(None,) dtype=int64>,

[INFO 23-07-28 16:02:31.4837 PDT kernel.cc:773] Start Yggdrasil model training
[INFO 23-07-28 16:02:31.4851 PDT kernel.cc:774] Collect training examples
[INFO 23-07-28 16:02:31.4851 PDT kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 23-07-28 16:02:31.4880 PDT kernel.cc:393] Number of batches: 2
[INFO 23-07-28 16:02:31.4881 PDT kernel.cc:394] Number of examples: 1242
[INFO 23-07-28 16:02:31.4921 PDT data_spec_inference.cc:305] 1242 item(s) have been pruned (i.e. they are considered out of dictionary) for the column nvidia_smi_log.timestamp (0 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
[INFO 23-07-28 16:02:31.4926 

Model trained in 0:00:00.159634
Compiling model...
Model compiled.


In [98]:
model_1.compile(metrics=["accuracy"])
evaluation = model_1.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

2/2 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 1.0000

loss: 0.0000
accuracy: 1.0000


In [101]:
model_1.make_inspector().variable_importances()

{'INV_MEAN_MIN_DEPTH': [("nvidia_smi_log.gpu.fb_memory_usage.free" (1; #12),
   0.5637331663012846),
  ("nvidia_smi_log.gpu.fb_memory_usage.used" (1; #13), 0.5560704355885083),
  ("nvidia_smi_log.gpu.bar1_memory_usage.free" (1; #3), 0.5545286506469506),
  ("nvidia_smi_log.gpu.bar1_memory_usage.used" (1; #5), 0.5429864253393671),
  ("nvidia_smi_log.gpu.clocks.sm_clock" (1; #8), 0.5011135857461034),
  ("nvidia_smi_log.gpu.pci.tx_util" (1; #20), 0.4946413849958789),
  ("nvidia_smi_log.gpu.clocks.graphics_clock" (1; #6), 0.4932858317347225),
  ("nvidia_smi_log.gpu.clocks.video_clock" (1; #9), 0.48926338678988923),
  ("nvidia_smi_log.gpu.utilization.memory_util" (1; #29), 0.4884667571234742),
  ("nvidia_smi_log.gpu.temperature.memory_temp" (1; #27), 0.4883342376559963)],
 'NUM_AS_ROOT': [("nvidia_smi_log.gpu.fb_memory_usage.free" (1; #12), 78.0),
  ("nvidia_smi_log.gpu.bar1_memory_usage.free" (1; #3), 72.0),
  ("nvidia_smi_log.gpu.fb_memory_usage.used" (1; #13), 72.0),
  ("nvidia_smi_log.gp

In [109]:
# using a subset of important features
feature_1 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.fb_memory_usage.free")
feature_2 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.fb_memory_usage.used")
feature_3 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.bar1_memory_usage.free")
feature_4 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.bar1_memory_usage.used")
feature_5 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.clocks.sm_clock")
feature_6 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.pci.tx_util")
feature_7 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.clocks.graphics_clock")
feature_8 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.utilization.memory_util")
feature_9 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.clocks.video_clock")
feature_10 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.temperature.memory_temp")


all_features = [feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7, feature_8]
model_2 = tfdf.keras.GradientBoostedTreesModel(
    features=all_features, exclude_non_specified_features=True)

model_2.compile(metrics=["accuracy"])
model_2.fit(train_ds, validation_data=test_ds)

print(model_2.evaluate(test_ds, return_dict=True))

Use /var/folders/wx/r87f739d16n6_b4_57_dnxdw0000gn/T/tmpah1goii9 as temporary training directory
Reading training dataset...


[WARNING 23-07-28 17:11:25.7484 PDT gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-07-28 17:11:25.7485 PDT gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-07-28 17:11:25.7485 PDT gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


Training dataset read in 0:00:00.320425. Found 1242 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(1242, shape=(), dtype=int32)
Validation dataset read in 0:00:00.288961. Found 1242 examples.
Training model...
Model trained in 0:00:00.561252
Compiling model...


[INFO 23-07-28 17:11:26.9182 PDT kernel.cc:1243] Loading model from path /var/folders/wx/r87f739d16n6_b4_57_dnxdw0000gn/T/tmpah1goii9/model/ with prefix 7c7aa3c95ddf4b8a
[INFO 23-07-28 17:11:26.9369 PDT quick_scorer_extended.cc:903] The binary was compiled without AVX2 support, but your CPU supports it. Enable it for faster model inference.
[INFO 23-07-28 17:11:26.9376 PDT abstract_model.cc:1312] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 23-07-28 17:11:26.9376 PDT kernel.cc:1075] Use fast generic engine


Model compiled.
2/2 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - accuracy: 1.0000
{'loss': 0.0, 'accuracy': 1.0}


In [13]:
# alexa dataset 

alexa = pd.read_csv("alexa-test.csv")

In [3]:
# reading existing and new logs 

existing = pd.read_csv("dns-dataset - existing.csv")
new_bad = pd.read_csv("dns-dataset - new-domains-bad.csv")
new_good = pd.read_csv("dns-dataset - new-domains-good.csv")
same = pd.read_csv("dns-dataset - same-domains.csv")

In [4]:
new_bad.head()

,Domains
0,facebook.com
1,anishka.com
2,zoom.us
3,cryptojacking.com
4,youtube.com


In [75]:
def checkDomains(existing, new):
    for index in existing.index:
        value = existing[existing.columns[0]].iloc[index]
        if (new.empty == True):
            pass
        elif (value in new['Domains '].values):
            new = new[new['Domains '] != value]
    new = new.reset_index()
    return (new)

In [76]:
def checkAlexa(new):
    if (new.empty):
        return False 
    else:
        for index in new.index: 
            if (new[new.columns[1]].iloc[index] not in alexa['google.com'].values):
                return True
    return False

In [79]:
df = checkDomains(existing, same)
print (checkAlexa(df))
df = checkDomains(existing, new_good)
print (checkAlexa(df))
df = checkDomains(existing, new_bad)
print (checkAlexa(df))

False
False
True
